In [1]:
#Import Libraries
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

In [3]:
#Create the client
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential("your_github_token_here"),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=14, completion_tokens=8) cached=False logprobs=None thought=None


In [5]:
#Define the function
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    # Make the search case-insensitive
    city_key = next((key for key in destinations.keys() if key.lower() == city.lower()), None)
    
    if city_key:
        return city_key, destinations[city_key]
    else:
        return city, "City not found"

# Example usage:
city, status = vacation_destinations("Barcelona")
print(f"How about visiting {city}? It's currently {status} there!")

# Test case-insensitive search
city2, status2 = vacation_destinations("barcelona")
print(f"How about visiting {city2}? It's currently {status2} there!")

How about visiting Barcelona? It's currently Available there!
How about visiting Barcelona? It's currently Available there!


In [6]:
#Define the function tool
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

In [7]:
#Define the Agent
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

In [8]:
#Run the Agent
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=74, completion_tokens=18), metadata={}, created_at=datetime.datetime(2025, 6, 27, 5, 3, 31, 280618, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_kMkTN2eBRNEUgR971f8gKDfN', arguments='{"city":"Tokyo"}', name='vacation_destinations')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 27, 5, 3, 31, 289635, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content="('Tokyo', 'Unavailable')", name='vacation_destinations', call_id='call_kMkTN2eBRNEUgR971f8gKDfN', is_error=False)], type='ToolCallExecutionEvent')]
source='assistant' models_usage=RequestUsage(prompt_tokens=65, completion_tokens=47) metadata={} created_at=datetime.datetime(2025, 6, 27, 5, 3, 33, 45797, tzinfo=datetime.timezone.utc) content='It seems that Tokyo is currently unavailable for travel. However, I can help you explo